In [ ]:
from IPython.display import Markdown, HTML
import matplotlib.pyplot as plt
%matplotlib inline

# Overcoming Car Troubles with Q-Learning
---

<div class="alert alert-danger">
<p>So, we gave two lectures in preparation for this workshop. You can find them on GitHub and Google Slides.</p>
<ol>
    <li>Can Machines Learn from Experience? (RL P1) [GitHub][rlp1-gh], [Google Slides][rlp1-gs]</li>
    <li>Can Machines Learn from Experience? (RL P2) [GitHub][rlp2-gh], [Google Slides][rlp2-gs]</li>
</ol>
</div>

[rlp1-gh]: https://github.com/ucfsigai/meetings/blob/master/sp18/07-lec-reinforcement-learning-p1.pdf
[rlp1-gs]: https://docs.google.com/presentation/d/1wHPmkonCFCsLBIBLZDYvxHs6EeW6vLDsXCxNOq0eNlY
[rlp2-gh]: https://github.com/ucfsigai/meetings/blob/master/sp18/08-lec-reinforcement-learning-p2.pdf
[rlp2-gs]: https://docs.google.com/presentation/d/1x1Mi_bhlGmNKQ7brraWJUMG7B9Y_WqU9970ni-ic5FY

---
## Markov Decision Processes (MDPs)

MDPs are, informally, a set of states which are contected together in some way. Each of those connections have some probability of being the next action you take, based on your current state. **e.g.** Imagine you're at an intersection, there's some probability that you'll proceed straight ahead, move to the left, move to the right, move backward, or do nothing (hopefully nothing more :p). That's a single step in a MDP.

You can find a [visual here][mdp-d3]; make sure, if you modify the transition matrix, such that each row sums to 1.

More formally... MDPs are a "5-tuple":

- $S$, a **_finite_** set of states
- $A$, a **_finite_** set of actions
- $P_a(s, s') = Pr(s_{t+1} = s' | s_t = s, a_t = a)$, the probability that an action $a$, in state $s$, at time $t$ will lead to state $s'$ in time $t + 1$
- $R_a(s, s')$, is the immediate reward received after transitioning from $s \rightarrow s'$, due to action $a$
- $\gamma \in [0, 1]$ is the discount factor, which affects an agent's emphasis on future rewards.

[mdp-d3]: http://localhost:19972/notebooks/sp18/assets/markov/index.html

We make these choices based on a policy ($\pi$), which maps states to actions. More specifically, we map the **_current state_** to some action. We can do this because of the Markov Property.
$$ \pi(s): S \rightarrow A $$

---
## Policy and Value Iteration

In [RL P2][rlp2-gs], we talked about Policy and Value Iteration. Some of the key snippets you need to know include...

Policy and Value Iteration (PVI) are considered model-based learning algorithms. Model-based learning means that the agent knows the MDP used to model the environment, before it begins exploring. This allows the agent to plan its actions within the given environment before taking any actions (this is often referring to as offline learning).


[rlp2-gs]: https://docs.google.com/presentation/d/1x1Mi_bhlGmNKQ7brraWJUMG7B9Y_WqU9970ni-ic5FY

### Slight Trangent: Value Function $\rightarrow V(s)$

Simply, the Value Function (typically denoted by $V(s)$), is a measure of "how good is this state?" The formulation of $V(s)$ is...

$$ V^\pi(s) = E[\Sigma_{i=1}^T \gamma^{i-1} \cdot r_i] \quad \forall s \in S $$

But... like with most things, there's an _Optimal Value Function_, which presents the best value among its peers.

$$ V^*(s) = max_\pi V^\pi(s) \quad \forall s \in S $$

Knowing this, we can also determine the _Optimal Policy_! :D

$$ \pi^* = arg max_\pi V^\pi(s) \quad \forall s \in S $$

### More Tangets: The Q-Function $\rightarrow Q(s)$

The Q-Function (also in RL P2), is a measure of "how good is it for me to pick action $a$ while in state $s$?" This differs from the Value Function $V(s)$ mostly in that $Q(s, a)$ considers the state **_and_** possible action(s), while $V(s)$ simply considers the current state.

Due to their similarity, we can actually restate our _Optimal Value Function_ as:

$$ V^*(s) = max_a Q^*(s, a) \quad \forall s \in S $$

So, if we know $Q^*(s, a)$ (the optimal Q-Function), we can not only extract the Optimal Value Function, but we can also derive the Optimal Policy! XD

$$ \pi^* = arg max_a Q^*(s, a) \quad \forall s \in S $$

---
## The fabled (or soon to be) Bellman Equations

The [equations][bellman-eqn] are based on a variety of discoveries by [Richard Bellman][rb-wiki] in Optimization. Essentially, they model the idea that...
> The value of a decision is based on the payoff of some initial choices and the value of the remaining decisions that result from those initial choices.

**Tangent:** This is pertinent to Dynamic Programming, which is all about optimization through breaking down a single problem into various subproblems.

[rb-wiki]: https://en.wikipedia.org/wiki/Richard_E._Bellman
[bellman-eqn]: https://en.wikipedia.org/wiki/Bellman_equation

\begin{align}
Q^*(s, a) &= R(s, a) + \gamma E_{s'}[V^*(s')] \\
Q^*(s, a) &= R(s, a) + \gamma \Sigma_{s' \in S}p(s'|s, a)V^*(s')
\end{align}

We know, from earlier, though... 

\begin{align}
V^*(s) &= max_a Q^*(s, a) \\
V^*(s) &= max_a [R(s, a) + \gamma \Sigma_{s' \in S}p(s'|s, a)V^*(s')]
\end{align}

Ultimately, PVI (Policy and Value Iteration) rely on these equations to compute $V^*(s)$.

## Value Iteration (VI)

Simply, VI computes the optimal state-value by iteratively updating $Q(s, a)$ and $V(s)$ until they converge. A perk of VI is that its guaranteed t oconverge to an optimal value.

## Policy Iteration (PI)

Something worth noting, though, is that there are cases where the policy converges before the value function does.  Also note that the agent only cares about finding the optimal policy. $V(s)$ must converge, as with VI; but improves the value function based on updates to the policy. PI, like VI, is guaranteed to converge, but in some cases may take require fewer iterations to do so.

## Notes on Value and Policy Iteration

- They're both used for offline planning (offline learning).
- Policy Iteration is computationally efficient on the path to convergence, but each iteration is more computationally expensive than Value Iteration.

---

---

# Q-Learning

A model-free learning algorithm, which relies on this equation:

\begin{align}
Q(s, a) &= (1 - \alpha) \cdot Q(s, a) + \alpha Q_{obs}(s, a) \\
Q_{obs}(s, a) &= r(s, a) + \gamma \cdot max_{a'} Q(s', a')
\end{align}

---
## We Qan almost learn. Just need some setup.

We're going to be using a toolkit called [Gym][site:openai/gym] [(on GitHub)][openai/gym], which is built by [OpenAI][site:openai]. Gym is an attempt at creating semi-standardized environments in which agents can be tested and have their results compared more effectively.

If you're interested in learning more about Gym, check out their [documentation][site:openai/gym/docs], it's a bit sparse; but should be enough to get you started, at least with using their prefab [environments][site:openai/gym/docs#env].

[site:openai]: https://openai.com/
[site:openai/gym]: https://gym.openai.com
[openai/gym]: https://github.com/openai/gym
[site:openai/gym/docs]: https://gym.openai.com/docs/
[site:openai/gym/docs#env]: https://gym.openai.com/docs/#environments

[Mountain Car](gym/mountaincar)

<video width="500" height="250">
    <source src="https://gym.openai.com/v2018-02-21/videos/MountainCar-v0-270f34b9-f23e-4d95-a933-4c902b4f4435/original.mp4" type="video/mp4">
</video>

In [ ]:
import numpy as np
import gym

In [ ]:
env_name = 'MountainCar-v0'
env = gym.make(env_name)
env.seed(0)
np.random.seed(0)

In [ ]:
display(Markdown("Action Space: `{}`".format(env.action_space)))

In [ ]:
display(Markdown("Observation Space: `{}`".format(env.observation_space)))

In [ ]:
display(Markdown("`env_lo` = `{}`".format(env.observation_space.low)))
env_lo = env.observation_space.low
display(Markdown("`env_hi` = `{}`".format(env.observation_space.high)))
env_hi = env.observation_space.high

Discounting factor

In [ ]:
gamma = 1.0

Learning rates

In [ ]:
lr_ini = 1.0
lr_min = 0.003

Probability of not exploring

In [ ]:
epsilon = 0.02

Episodic and temporal limits

In [ ]:
max_iter = 10000
max_time = 10000

Number of states...

In [ ]:
n_states = 40

Given the exploration we did earlier, we know that our `env.action_space` is a [`Discrete`][openai/gym/discrete] space; based on the definition of the [`Discrete`][openai/gym/discrete.n] space... we can use that to define our `n_actions` (number of actions).

[openai/gym/discrete]: https://github.com/openai/gym/blob/master/gym/spaces/discrete.py
[openai/gym/discrete.n]: https://github.com/openai/gym/blob/master/gym/spaces/discrete.py#L12

In [ ]:
n_actions = env.action_space.n

In [ ]:
env_dx = (env_hi - env_lo) / n_states

## On to Q-Learning!

In [ ]:
def observations2states(env, obs):
    a = int((obs[0] - env_lo[0]) / env_dx[0])
    b = int((obs[1] - env_lo[1]) / env_dx[1])

    return a, b

In [ ]:
## initialize the Q table
q_table = np.zeros((n_states, n_states, n_actions))

In [ ]:
for episode in range(max_iter):

    obs = env.reset()
    total_reward = 0

    ## learning rate is decreased at each step
    lr = max(lr_min, lr_ini * (0.85 ** (episode // 100)))

    for _ in range(max_time):
        ## make an attempt, and retribe an observation
        a, b = observations2states(env, obs)

        logits     = q_table[a][b]
        logits_exp = np.exp(logits)

        weighted_probs = logits_exp / np.sum(logits_exp)

        explore = np.random.uniform(0, 1) < epsilon
        distrib = None if explore else weighted_probs
        action  = np.random.choice(env.action_space.n, p=distrib)

        obs, reward, done, _ = env.step(action)
        total_reward += reward
        
        ## we move into the "next timestep", so what is "prev_action" is 
        ## the Q value of the action taken above
        a_, b_ = observations2states(env, obs)

        ## prior action we took
        prev_action = q_table[a][b][action]
        ## action we might take before of prior action
        next_action = q_table[a_][b_]

        ## Q(s, a) = (1 - \alpha) * Q(s, a) 
        ##           + \alpha * [r(s, a) + \gamma * max{Q(s', a')}]
        ## some voodoo later...
        ## Q(s, a) = Q(s, a) + \alpha * [r(s, a) + \gamma * max{Q(s', q')} - Q(s, a)]
        q_table[a][b][action] = prev_action + lr * (reward + gamma * np.max(next_action) - prev_action)

        if done:
            break

    if episode % 100 == 0 or episode == max_iter - 1:
        print('iter: {0:5d} | reward: {1:5.5f}'.format(episode, total_reward))

In [ ]:
def run_episode(env, policy=None, render=False):

    obs = env.reset()
    
    total_reward = 0

    ## note: `step_idx` for the discounting factor is initialized here
    for step_idx in range(max_time):
        
        if render:
            plt.imshow(env.render(mode='rgb_array'))
            display.clear_output(wait=True)
            display.display(plt.gcf())

        a, b = observations2states(env, obs)
        action = env.action_space.sample() if policy is None else policy[a][b]
        
        obs, reward, done, _ = env.step(action)
        total_reward += (gamma ** step_idx) * reward
        
        if done:
            return total_reward

In [ ]:
solution_policy = np.argmax(q_table, axis=2)
np.set_printoptions(threshold=np.inf, linewidth=120)
print(solution_policy)

## Score the solutions
solution_policy_scores = [run_episode(env, policy=solution_policy) for _ in range(100)]

print("mean(reward): {0:5.5f}".format(np.mean(solution_policy_scores)))

## Visualize actions based on the best solution
run_episode(env, policy=solution_policy, render=True)